In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
%matplotlib inline

In [ ]:
class Wine:
    def __init__(self, df, wine_name: str):
        self.df=df
        self.wine_name=wine_name
        
    def get_common_analise(self):
        print('df.head()\n')
        display(self.df.head())
        print('\n\n df.info()\n')
        display(self.df.info())
        print(f'\n\n Количество дубликатов={self.df.duplicated().sum()}')
        print('\n\n self.df.describe().T\n')
        display(self.df.describe().T)
        
    def histplots_kdes(self):    
        fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(15, 12))

        # Цикл для построения гистограмм для каждого числового признака
        for i, col in enumerate(self.df.columns[:-1]):
            x = i // 3
            y = i % 3
            values, bin_edges = np.histogram(self.df[col],
                                            range=(np.floor(self.df[col].min()), np.ceil(self.df[col].max())))

            graph = sns.histplot(data=self.df, x=col, bins=bin_edges, kde=True, ax=ax[x, y], color='red')
            ax[x, y].set_xlabel(col, fontsize=15)
            ax[x, y].set_ylabel('Количество', fontsize=12)
            ax[x, y].set_xticks(np.round(bin_edges, 1))
            ax[x, y].set_xticklabels(ax[x, y].get_xticks(), rotation=45)
            ax[x, y].grid(color='lightgrey')

            for j, p in enumerate(graph.patches):
                ax[x, y].annotate('{}'.format(p.get_height()), (p.get_x() + p.get_width() / 2, p.get_height() + 1),
                                ha='center', fontsize=10, fontweight="bold")

            textstr = '\n'.join((
                r'$\mu=%.2f$' % self.df[col].mean(),
                r'$\sigma=%.2f$' % self.df[col].std()
            ))
            ax[x, y].text(0.75, 0.9, textstr, transform=ax[x, y].transAxes, fontsize=12, verticalalignment='top',
                        color='white', bbox=dict(boxstyle='round', facecolor='#ff826e', edgecolor='white', pad=0.5))

        ax[1,2].axis('off')
        plt.suptitle(f'Распределение характеристик {self.wine_name} вин', fontsize=20)
        plt.tight_layout()
        plt.subplots_adjust(top=0.92)
        plt.show()
        
     
    def boxenplots(self):
        fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(15, 12))

        # Цикл для построения боксплота для каждого числового признака
        for i, col in enumerate(self.df.columns[:-1]):
            x = i // 3
            y = i % 3
            values, bin_edges = np.histogram(self.df[col],
                                            range=(np.floor(self.df[col].min()), np.ceil(self.df[col].max())))

            graph =sns.boxenplot(data=self.df, x=col,ax=ax[x, y], color='blue')
            ax[x, y].set_xlabel(col, fontsize=15)
            ax[x, y].set_ylabel('Количество', fontsize=12)
            ax[x, y].set_xticks(np.round(bin_edges, 1))
            ax[x, y].set_xticklabels(ax[x, y].get_xticks(), rotation=45)
            ax[x, y].grid(color='lightgrey')

            textstr = '\n'.join((
                r'$\mu=%.2f$' % self.df[col].mean(),
                r'$\sigma=%.2f$' % self.df[col].std()
            ))
            ax[x, y].text(0.75, 0.9, textstr, transform=ax[x, y].transAxes, fontsize=12, verticalalignment='top',
                        color='white', bbox=dict(boxstyle='round', facecolor='#ff826e', edgecolor='white', pad=0.5))

        ax[1,2].axis('off')
        plt.suptitle('Боксплоты для характеристик красных вин', fontsize=20)
        plt.tight_layout()
        plt.subplots_adjust(top=0.92)
        plt.show()
    
    def horizontal_barplot(self, ax, color):
        # Вычисление процентных значений частоты красных вин
        value_counts=self.df.quality.value_counts(normalize=True).mul(100).sort_values()
        # Построение столбчатой диаграммы
        value_counts.plot(kind='barh', width=0.5, color=color, ax=ax)
        for index, value in enumerate(value_counts):
            ax.text(value, index, str(round(value, 1))+'%', fontsize=10)
            ax.set_xlabel('Качество красных вин', fontsize=10)
            
    def mean_assessment(self):
        print(f'Средняя оценка {self.wine_name} вин={self.df.quality.mean().round(2)}')
    
    def two_dementional_analisis(self):
        #Построим кривые взаимосвязи каждого признака с целевой переменной - качеством вина для красных вин
        sns.set_palette(['#ff826e','red', 'blue', 'green', 'black', 'yellow', 'orange', 'pink', 'brown', 'purple', 'white'])
        fig,ax = plt.subplots(len(self.df.columns)-1, 2, figsize=(15, 30), gridspec_kw={'width_ratios':[1,2]})
        # Цикл для каждого численного признака для создания линейчатых диаграмм и графиков kde
        for i,col in enumerate(self.df.columns[:-1]):
        # Гистограмма, показывающая среднее значение признака для каждой целевой категории
            graph =sns.barplot(data=self.df, x='quality', y=col, ax=ax[i,0])
            # График KDE, показывающий распределение признака для каждой целевой категории
            for n in range(3,9):
                sns.kdeplot(data=self.df[self.df['quality']==n],x=col, fill=True, linewidth=2,ax=ax[i,1],label=str(n))
            ax[i,1].set_yticks([])
            ax[i,1].legend(title=f'Качество {self.wine_name} вин', loc='upper right')

        # средние значения в гистограмму
            for count in graph.containers:
                graph.bar_label(count, fmt='         %.3g')

        plt.suptitle(f'Количественные признаки {self.wine_name} вин в сравнении с качеством', fontsize=22, x=0.5, y=1.0)
        plt.tight_layout()
        plt.show();
        
        correlation=self.df.corr()
        
        plt.figure(figsize=(12, 4))
        sns.heatmap(correlation.T, annot=True, cmap='coolwarm', center=0, linewidth=1)
        plt.title('Корреляция каждого признака со всеми признаками и целевой переменной quality', fontsize=16)
        plt.show;

        print(f'\n\n Посмотрим коэффициенты корреляции признаков с качеством для {self.wine_name} вин\n')
        correl=correlation['quality']
        display(correl) 
        
        for i, c in enumerate(correl[:-1]):
            if 0<c<0.3:
                print(f'Слабая положительная зависимость с {self.df.columns[i]} = {round(c, 2)}')
            elif -0.3<=c<=0:
                print(f'Слабая отрицательная зависимость с {self.df.columns[i]} = {round(c, 2)}')
            elif 0.3<=c<0.7:
                print(f'Средняя положительная зависимость с {self.df.columns[i]} = {round(c, 2)}')
            elif -0.7<=c<-0.3:
                print(f'Средняя отрицательная зависимость с {self.df.columns[i]} = {round(c, 2)}')
            elif c>=0.7:
                print(f'Сильная положительная зависимость с {self.df.columns[i]} = {round(c, 2)}') 
            else:
                print(f'Сильная отрицательная зависимость с {self.df.columns[i]} = {round(c, 2)}')
                
        print('\n\n')
        
        